In [19]:
# beautifulsoup
import requests
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep

In [20]:
# 時論公論(トップの10記事のみ)
url = 'https://www.nhk.jp/p/ts/4V23PRP3YR/list/?pastOffset=10'
# セットアップ
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)

【流れ】
記事をクリック→もし「続きを読む」があったらクリック→タイトルを取得→記事本文を取得

In [21]:
title_list = []

res = requests.get(url)

wait = WebDriverWait(driver, 10)

for num in range(1,11):
    
    article_x_path = f'//*[@id="past-pg"]/ul/li[{num}]/div/div[3]/div[1]/a/div[1]/h3'
    wait.until(EC.presence_of_element_located((By.XPATH, article_x_path)))
    select_article = driver.find_element(By.XPATH, article_x_path)
    select_article.click()
    
    more_read_x_path = '//*[@id="article-0"]/div[3]/div[2]/a'
    
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, more_read_x_path)))
        more_read_button = driver.find_element(By.XPATH, more_read_x_path)
    
    except: 
        driver.back()
        sleep(1)
        continue
    
    more_read_button.click()
    sleep(2)
    
    # Seleniumでページのソースを取得(ボタンを押したあと＝レンダリング後のURLを取得したいのでSeleniumでurl取得)
    page_source = driver.page_source
    
    # BeautifulSoupで解析
    soup = BeautifulSoup(page_source, 'html.parser')
    # soup = BeautifulSoup(c, 'html.parser') # このページのタグを一斉に取得
    
    main = soup.find('article')
    # タイトル取得
    title = main.find('h1').text.strip()
    title_list.append(title)
    
    paragraphs = main.find_all('p')
    main_text = '\n'.join([p.text.strip() for p in paragraphs])
    
    with open(f"data/{title}.txt", 'w', encoding='utf-8') as file:
        file.write(main_text)
    
    driver.back()
    sleep(1)

# ブラウザを閉じる
driver.quit()
print("finish")

finish


In [27]:
import pandas as pd
pd.DataFrame(title_list).to_csv('data/時論公論_title.csv')